# 15天入门Python3

CopyRight by 黑板客  
转载请联系heibanke_at_aliyun.com

**上节作业**

1. day05文件夹下有一些英文小说文件，用已经学过的知识编写程序统计出每个文件里单词出现频率最高的前10大单词是什么？
2. 你会发现只有001这个文件最好用，其他文件报错。其他文件的问题在哪里。（选做）


In [ ]:
%load day05/tongji.py  # 从参考资料里下载day05.zip，然后解压到Code目录下即可。

## day06：装饰器—给函数打扮一下

1. <a href="#1">闭包closure</a>
2. <a href="#2">变量作用域</a>
3. <a href="#3">装饰器</a>
4. <a href="#4">作业</a>

## <a name="1">闭包closure</a>

1. 嵌套函数
2. 内部函数使用外部变量(通常是外部函数的参数)
3. 外部函数返回内部函数

In [ ]:
from __future__ import print_function


# closure
def pow_x(x):
    def echo(value):
        return value**x
    return echo

lst = [pow_x(2), pow_x(3), pow_x(4)]
for p in lst:
    print("closure",p(2), p.__closure__[0].cell_contents)


In [ ]:
# 闭包使用的变量注意事项
def count():
    fs = []
    for i in range(1, 4):
        def f():
             return i*i
        fs.append(f)
    return fs

f1, f2, f3 = count()
print(f1(), f2(), f3())

In [ ]:
# -*- coding: utf-8 -*-
from __future__ import print_function

import random
origin = [0, 0] # 坐标系统原点 

def create(pos): 
    def player(direction,step): 
        """
        direction: ［x, y]
          0: 在这个方向不走
          1: 朝正方向走
         -1: 朝反方向走
        step: [x, y]
          朝两个方向各走多远
        """  
        pos[0] = pos[0] + direction[0]*step[0] 
        pos[1] = pos[1] + direction[1]*step[1]
        return pos 
    return player 

players = [create(origin[:]) for i in range(5)] # 创建5个player，起点都为原点 

#每个player随机走10步，看最终各自的位置。
for i in range(10):
    print("========== 第%d步 =========="%(i+1))
    for name, player in enumerate(players):
        # enumerate可以在遍历列表时同时产生元素的位置，这里把位置当作player的名字
        # 产生随机方向和随机步长
        direction = [random.choice([-1,1]), random.choice([-1,1])]
        step = [random.randint(0,20), random.randint(0,20)]
        print(name,player(direction, step))


### 问题加练习

1. 如果走10000步，100000步呢，最终每个player的位置会往哪个方向走？
2. 为什么？如果想让它往某个方向走，需要怎么修改程序？
3. 如果想把走的轨迹图示出来，可以使用第三方的库matplotlib。感兴趣可以自己试试。在我另一门课《用Python做深度学习1》里有介绍。

## <a name="2">变量作用域</a>

In [ ]:
# Build-in Python #######################################
#
#        ** Global **************************************
#        **
#        **        $$ Enclosing function local $$$$$$$$$$
#        **        $$ 
#        **        $$         @@ local function @@@@@@@@@
#        **        $$         @@  
#        **        $$         @@ 
#        **        $$         @@ 
#        **        $$         @@ 

In [ ]:
from __future__ import print_function

# 1. Global
g_X = 9 
def f1(s):
    # global g_X
    g_X = s
    # 2. Global or enclosing 
    def f2():
        # 3. enclosing or local
        # g_X=7
        print("in f2()", g_X)
    print("in f1()",g_X)
    return f2

if __name__=="__main__":
    a = f1(6)
    b = f1(8)
    print("in main",g_X)
    
    a()
    b()
    

## <a name="3">装饰器</a>

In [ ]:
# 装饰器也是函数

def func1():  
    print("func1() called.") 
    
def decorator(f):  
    print("before f() called.") 
    return f

def func2():  
    print("func2() called.")

f1 = decorator(func1)
f2 = decorator(func2)

f1()
f2()

In [ ]:
# 1. 最简单的装饰器

@decorator
def func1():  
    print("func1() called.")  

def decorator(f):  
    print("before f() called.")  
    return f

@decorator
def func2():  
    print("func2() called.")  

#func1()
#func2()

### 最简单装饰器存在问题
1. 装饰器会提前执行
2. 目标函数无法带参数
3. 目标函数调用后无法插入代码，只能在调用前插入代码

In [ ]:
# 2层改进
import time
def decorator(f):
    def _f(*arg, **kwarg):
        print("before f() called")
        start = time.clock()
        f(*arg,**kwarg)
        end = time.clock()
        print(end-start)
    return _f

@decorator
def func2():  
    print("func2() called.") 
    
func2()

### 2层装饰器存在问题
1. ~~@装饰器会提前执行~~
2. ~~目标函数无法带参数~~
3. ~~函数调用后无法插入代码~~
4. 装饰器无法带参数


In [ ]:
# 3层装饰器
import time

def time_cost(loopnum):  # 装饰器参数
    def decorator(f):    # f装饰的函数
        def _f(*arg, **kwarg):  # 装饰的函数调用时的参数
            """
            decorator_f
            """
            start = time.clock()
            for i in range(loopnum):
                f(*arg,**kwarg)
            end = time.clock()
            print(f.__name__,"run cost time is ",end-start)
        return _f

    return decorator

@time_cost(2)
def func2():
    """
    func2
    """
    print("func2() called.") 
    
func2()

print(func2.__name__)
help(func2)

这个三层，还有什么问题？

### functools.wraps

In [ ]:
# 3层装饰器
from functools import wraps
import time

def time_cost(loopnum):  # 装饰器参数
    def decorator(f):    # f装饰的函数
        @wraps(f)
        def _f(*arg, **kwarg):  # 装饰的函数调用时的参数
            """
            decorator_f
            """
            start = time.clock()
            for i in range(loopnum):
                f(*arg,**kwarg)
            end = time.clock()
            print(f.__name__,"run cost time is ",end-start)
        return _f

    return decorator

@time_cost(3)
def func2():  
    """
    func2
    """
    print("func2() called.") 
    
func2()

print(func2.__name__)
help(func2)

<a name="4">**作业：用装饰器完成穿衣服的例子**</a>

1. 用装饰器完成给人穿各种衣服的例子
2. 实现套装适合各种场景，比如：上班工作套装，运动套装，休闲套装等。

In [ ]:
from __future__ import print_function
from __future__ import unicode_literals
from day06.decorator_mode import wear_cloth, taozhuang

@wear_cloth("毛衣")
@wear_cloth("T恤")
@wear_cloth("裤子")
def person(name):
    print("装扮好的%s" % name)


@taozhuang(["衬衫","裤子","西装","皮鞋","领带"])
def work_person(name):
    print("装扮好的%s" % name)

@taozhuang(["运动衣", "运动裤","运动鞋"])
def sport_person(name):
    print("装扮好的%s" % name)



work_person("小明")